Reading data to DataFrame:

In [1]:
import pandas as pd

from sklearn.dummy import DummyClassifier          
from sklearn.linear_model import LogisticRegression

#success metric
from sklearn.metrics import accuracy_score

df = pd.read_csv("../input/polish_names.csv")
df.head()

,name,gender
0,Abdon,m
1,Abel,m
2,Abercjusz,m
3,Abraham,m
4,Absalon,m


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    1705 non-null   object
 1   gender  1705 non-null   object
dtypes: object(2)
memory usage: 26.8+ KB


In [6]:
df.sample(12)

,name,gender
579,Gościsław,m
495,Felicja,f
402,Drogomysł,m
559,Gilbert,m
1066,Otton,m
911,Marcjan,m
732,Klarysa,f
329,Daria,f
997,Nadzieja,f
790,Laurencja,f


Verify the number of values based on the 'gender' column:

In [7]:
df['gender'].value_counts()

m    1033
f     672
Name: gender, dtype: int64

Transform m=>1 f=>0

In [8]:
def transform_string_into_number(string):
    return int(string == 'm')
    
df['gender'].head().map( transform_string_into_number )

0    1
1    1
2    1
3    1
4    1
Name: gender, dtype: int64

In [9]:
df['target'] = df['gender'].map( lambda x: int(x == 'm') )
df.head(10)

,name,gender,target
0,Abdon,m,1
1,Abel,m,1
2,Abercjusz,m,1
3,Abraham,m,1
4,Absalon,m,1
5,Achacjusz,m,1
6,Achacy,m,1
7,Achilles,m,1
8,Ada,f,0
9,Adalbert,m,1


New feature - length name:

In [10]:
df['len_name'] = df['name'].map(lambda x: len(x))

Train model:

In [11]:
X = df[ ['len_name'] ].values
y = df['target'].values

model = DummyClassifier()
model.fit(X, y)
y_pred = model.predict(X)

/opt/conda/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "


In [12]:
df['gender_pred'] = y_pred
df['gender_pred'].value_counts()

1    1037
0     668
Name: gender_pred, dtype: int64

In [13]:
df[ df.target != y_pred ].shape

(822, 5)

In [14]:
accuracy_score(y, y_pred)

0.5178885630498534

Logistic Regression

In [18]:
def train_and_predict_model(X, y, model, success_metric=accuracy_score):
    model.fit(X, y)
    y_pred = model.predict(X)
    
    print("Distribution:")
    print( pd.Series(y_pred).value_counts() )
    
    return success_metric(y, y_pred)

New feature: vowels:

In [19]:
vowels = ['a', 'ą', 'e', 'ę', 'i', 'o', 'u', 'y']

def how_many_vowels(name):
    count = sum( map(lambda x: int(x in vowels), name.lower()) )
    
    return count

#how_many_vowels('Jana')

df['count_vowels'] = df['name'].map(how_many_vowels)
train_and_predict_model(df[['len_name', 'count_vowels'] ], y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1082
0     623
dtype: int64


0.7143695014662756

In [20]:
def first_is_vowel(name):
    return name.lower()[0] in vowels

#first_is_vowel('Ada')

df['first_is_vowel'] = df['name'].map(first_is_vowel)

train_and_predict_model(df[['len_name', 'first_is_vowel'] ], y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1705
dtype: int64


0.6058651026392962

In [23]:
df['first_letter'] = df['name'].map(lambda x: x.lower()[0])
df['first_letter_cnt'] = df['first_letter'].factorize()[0]

X = df[['len_name', 'count_vowels', 'first_is_vowel', 'first_letter_cnt'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1106
0     599
dtype: int64


0.7296187683284457

In [25]:
def get_all_vowels(name):
    all_vowels = [letter for letter in name.lower() if letter in vowels]
    
    return ''.join(all_vowels)
    
#get_all_vowels('Sławomir')

df['all_vowels'] = df['name'].map(get_all_vowels)
df['all_vowels_cnt'] = pd.factorize(df['all_vowels'])[0]


X = df[['len_name', 'count_vowels', 'first_is_vowel', 'first_letter_cnt', 'all_vowels_cnt'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs'))

Distribution:
1    1129
0     576
dtype: int64


0.73841642228739

In [26]:
def get_all_consonants(name):
    all_consonants = [letter for letter in name.lower() if letter not in vowels]
    
    return ''.join(all_consonants)
    
#get_all_consonants('Sławomir')

df['all_consonants'] = df['name'].map(get_all_consonants)
df['all_consonants_cnt'] = pd.factorize(df['all_consonants'])[0]

X = df[['len_name', 'count_vowels', 'first_is_vowel', 'first_letter_cnt', 'all_consonants_cnt'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs', max_iter=200))

Distribution:
1    1111
0     594
dtype: int64


0.7313782991202346

In [27]:
def last_is_vowel(name):
    return name.lower()[-1] in vowels

#last_is_vowel('Ada')

df['last_is_vowel'] = df['name'].map(last_is_vowel)

X = df[['last_is_vowel'] ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs', max_iter=200))

Distribution:
1    964
0    741
dtype: int64


0.9524926686217009

In [28]:
feats = ['last_is_vowel', 'len_name', 'count_vowels', 'first_is_vowel', 'all_vowels_cnt', 'all_consonants_cnt']
X = df[ feats ]
train_and_predict_model(X, y, LogisticRegression(solver='lbfgs', max_iter=200))

Distribution:
1    964
0    741
dtype: int64


0.9524926686217009